<a href="https://colab.research.google.com/github/Sidhtang/30-day-ml-challenge-/blob/main/day_22_MAKING_TRANSFOMRES_EFFICIENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
#KNOWLEDGE DISTILLATION ,QUANTIZATION, PRUNING , AND GRAPH OPTIMIZATION

In [44]:
#ALL THESE TECHNIQUES WILL BE DONW THRI=OUGH OPEN NEURAL NETWORK eXCHANGE AND ORT

In [1]:
# INTENTION DETCTION CASE STUDY !
! pip install transformers






In [2]:
from transformers import pipeline
bert_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
pipe = pipeline("text-classification", model=bert_ckpt)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
query = """Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th in
Paris and I need a 15 passenger van"""
pipe(query)


[{'label': 'car_rental', 'score': 0.5490034222602844}]

In [48]:
#creating a performance benchmark
# model performance and latency (how a=fast model can deleiver the results),memory

In [49]:
# we  have deined an optim_type parameter to keep track of the different optimzation

class PerformanceBenchmark:
    def __init__(self, pipeline, dataset, optim_type="BERT baseline"):
        self.pipeline = pipeline
        self.dataset = dataset
        self.optim_type = optim_type

    def compute_accuracy(self):
        # We'll define this later
        pass

    def compute_size(self):
        # We'll define this later
        pass

    def time_pipeline(self):
        # We'll define this later
        pass

    def run_benchmark(self):
        metrics = {}
        metrics[self.optim_type] = self.compute_size()
        metrics[self.optim_type].update(self.time_pipeline())
        metrics[self.optim_type].update(self.compute_accuracy())
        return metrics


In [50]:
# lets check thee model acuracy by Clinic 50 datset
!pip install datasets

In [51]:
!pip uninstall pyarrow -y
!pip install pyarrow

Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0
  Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-17.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (39.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [18]:

#hide_output
from datasets import load_dataset

clinc = load_dataset("clinc_oos", "plus")


In [53]:
sample = clinc["test"][42]
sample



{'text': 'transfer $100 from my checking to saving account', 'intent': 133}

In [54]:
intents = clinc["test"].features["intent"]
intents.int2str(sample["intent"])

'transfer'

In [55]:
from datasets import load_metric
accuracy_score = load_metric("accuracy")

In [56]:
def compute_accuracy(self):
    """This overrides the PerformanceBenchmark.compute_accuracy() method"""
    preds, labels = [], []
    for example in self.dataset:
        pred = self.pipeline(example["text"])[0]["label"] # Indent this line
        label = example["intent"] # Indent this line
        preds.append(intents.str2int(pred)) # Indent this line
        labels.append(label) # Indent this line
    accuracy = accuracy_score.compute(predictions=preds, references=labels)
    print(f"Accuracy on test set - {accuracy['accuracy']:.3f}")
    return accuracy


In [57]:
# lets compute the size of our model by using the torch.sAVE FUNCTION
#  TO SEE WAHT IS stored in thr state_dict(model)
list(pipe.model.state_dict().items())[42]

('bert.encoder.layer.2.attention.self.value.bias',
 tensor([-2.7834e-02,  4.9434e-02,  8.3551e-02,  4.1092e-02,  6.0157e-01,
          1.1774e-01, -5.2112e-02, -6.5143e-02, -2.9358e-02, -4.2250e-02,
          7.9177e-02,  8.0409e-02,  2.9921e-03,  1.7816e-01, -5.0480e-02,
         -1.5634e-01, -2.1707e-02,  1.4381e-02,  2.5132e-02, -2.4110e-02,
         -1.9183e-01, -7.8657e-02,  5.0709e-02,  3.3632e-02, -3.1946e-02,
          1.1616e-01,  9.2720e-02, -1.1787e-01,  2.3233e-01, -1.2678e-02,
         -1.3138e-01, -4.0024e-02,  7.4823e-02, -5.4148e-02, -1.5184e-01,
         -7.4407e-02,  1.1559e-01,  8.2729e-02, -1.3787e-01,  8.3528e-02,
          1.2154e-01,  1.6880e-02, -5.6629e-02, -3.9295e-02,  5.3725e-02,
          6.8602e-02, -1.1294e-01,  4.4001e-02, -2.5884e-01,  1.6767e-01,
          1.8316e-01,  5.6272e-02, -3.6874e-02, -2.7938e-02, -9.3204e-02,
         -7.5239e-03,  4.1141e-02, -1.1542e-02, -9.9749e-02, -3.0910e-02,
          4.1398e-02, -4.4389e-02, -2.6279e-02,  7.2100e-02, 

In [58]:
import torch
torch.save(pipe.model.state_dict(), "model.pt")


In [59]:
import torch
from pathlib import Path
def compute_size(self):
 """This overrides the PerformanceBenchmark.compute_size() method"""
 state_dict = self.pipeline.model.state_dict()
 tmp_path = Path("model.pt")
 torch.save(state_dict, tmp_path)
 # Calculate size in megabytes
 size_mb = Path(tmp_path).stat().st_size / (1024 * 1024)
 # Delete temporary file
 tmp_path.unlink()
 print(f"Model size (MB) - {size_mb:.2f}")
 return {"size_mb": size_mb}


In [60]:
# lets  implement the time pipeline  so that we can
#time the average latency per query  so the time it takes
# to feed a text to query


In [5]:

from time import perf_counter

for _ in range(3):
    start_time = perf_counter()
    _ = pipe(query)
    latency = perf_counter() - start_time
    print(f"Latency (ms) - {1000 * latency:.3f}")

Latency (ms) - 151.100
Latency (ms) - 154.522
Latency (ms) - 156.409


In [9]:
#collecting latencies over many runs and then calculating hte avegge latency

import numpy as np

def time_pipeline(self, query="What is the pin number for my account?"):
    """This overrides the PerformanceBenchmark.time_pipeline() method"""
    latencies = []
    # Warmup
    for _ in range(10):
        _ = self.pipeline(query)
    # Timed run
    for _ in range(100):
        start_time = perf_counter()
        _ = self.pipeline(query)
        latency = perf_counter() - start_time
        latencies.append(latency)
    # Compute run statistics
    time_avg_ms = 1000 * np.mean(latencies)
    time_std_ms = 1000 * np.std(latencies)
    print(f"Average latency (ms) - {time_avg_ms:.2f} +\- {time_std_ms:.2f}")
    return {"time_avg_ms": time_avg_ms, "time_std_ms": time_std_ms}




In [10]:
# @title making models smalller through knowledge distillation
#For supervised tasks like fine-tuning, the main idea is to augment
#the ground truth labels with a distribution of “soft probabilities” from the teacher
#which provide complementary information for the student to learn from. For exam‐
#ple, if our BERT-base classifier assigns high probabilities to multiple intents, then this
#could be a sign that these intents lie close to each other in the feature space. By train‐
#ing the student to mimic these probabilities, the goal is to distill some of this



In [60]:
# we often soften the probabilities
# by applying thr hyperparameter
# to check how much knowldege is  lost when we approximate
# the probaility distribution this help us to define a
# knowledge a  distillaton loss
#nce we already have a fine-tuned BERT-base model, let’s see how we can use knowl‐
 #edge distillation to fine-tune a smaller and faster model. To do that we’ll need a way
 #to augment the cross-entropy loss with an LKD term


In [12]:
# @title  create a knowledge distillation trainer


from transformers import TrainingArguments

class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature

In [13]:
# When we instantiate DistillationTrainer we pass a
# teacher_model argument with a teacher that has already been fine-tuned on our task.
# Next, in the compute_loss() method we extract the logits from the student and
# teacher, scale them by the temperature, and then normalize them with a softmax
# before passing them to PyTorch’s nn.KLDivLoss() function for computing the KL
# divergence. One quirk with nn.KLDivLoss() is that it expects the inputs in the form
# of log probabilities and the labels as normal probabilities.
import torch
import torch.nn as nn
from transformers import Trainer
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    def compute_loss(self, model, inputs, return_outputs=False):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        inputs = inputs.to(device)
        outputs_stu = model(**inputs)
        # Extract cross-entropy loss and logits from student
        loss_ce = outputs_stu.loss
        logits_stu = outputs_stu.logits
        # Extract logits from teacher
        with torch.no_grad():
            outputs_tea = self.teacher_model(**inputs)
            logits_tea = outputs_tea.logits
        # Soften probabilities and compute distillation loss
        loss_fct = nn.KLDivLoss(reduction="batchmean")
        loss_kd = self.args.temperature ** 2 * loss_fct(
            F.log_softmax(logits_stu / self.args.temperature, dim=-1),
            F.softmax(logits_tea / self.args.temperature, dim=-1))
        # Return weighted student loss
        loss = self.args.alpha * loss_ce + (1. - self.args.alpha) * loss_kd
        return (loss, outputs_stu) if return_outputs else loss



In [19]:
# choosing a good student unutalisation
#so DistilBERT is a natural candidate to initialize the stu‐
 #dent with since it has 40% fewer parameters and has been shown to achieve strong
 # results on downstream tasks.
 #First we’ll need to tokenize and encode our queries, so let’s instantiate the tokenizer
 #from DistilBERT and create a simple tokenize_text() function to take care of the
 # preprocessing:
from transformers import  AutoTokenizer
student_ckpt = "distilbert-base-uncased" # Fixed typo here
student_tokenizer = AutoTokenizer.from_pretrained(student_ckpt)
def tokenize_text(batch):
 return student_tokenizer(batch["text"], truncation=True)
clinc_enc = clinc.map(tokenize_text, batched=True, remove_columns=["text"])
clinc_enc = clinc_enc.rename_column("intent", "labels")

Map:   0%|          | 0/15250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3100 [00:00<?, ? examples/s]

Map:   0%|          | 0/5500 [00:00<?, ? examples/s]

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
def compute_metrixs(pred):
  predictions,labels = pred
  predictions  = np.argmax(predictions,axis=1)
  return accuracy_score.compute(predictions=predictions,references=labels)


In [22]:
batch_size=48
finetuned_ckpt = "distilbert-base-uncased-finetuned-clinc"
student_training_args = DistillationTrainingArguments(
 output_dir=finetuned_ckpt, evaluation_strategy = "epoch",
 num_train_epochs=5, learning_rate=2e-5,
 per_device_train_batch_size=batch_size,
 per_device_eval_batch_size=batch_size, alpha=1, weight_decay=0.01,
 push_to_hub=True)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [23]:
id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

In [26]:

from transformers import AutoConfig

num_labels = intents.num_classes
student_config = (AutoConfig
                  .from_pretrained(student_ckpt, num_labels=num_labels,
                                   id2label=id2label, label2id=label2id))

NameError: name 'intents' is not defined